In [4]:
import boto3

# Connect to MinIO
s3 = boto3.client(
    's3',
    endpoint_url='http://localhost:9000',  # MinIO endpoint
    aws_access_key_id='admin',
    aws_secret_access_key='password',
)

# List all buckets
buckets = s3.list_buckets()
print("Buckets:")
for b in buckets['Buckets']:
    print(" ", b['Name'])

# List files in your ECG path
bucket_name = 'warehouse'
prefix = 'fs/ecg_raw/day=20251206/'

objects = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
print(f"\nFiles in {prefix}:")
if 'Contents' in objects:
    for obj in objects['Contents']:
        print(" ", obj['Key'])
else:
    print("No files found")


Buckets:
  warehouse

Files in fs/ecg_raw/day=20251206/:
  fs/ecg_raw/day=20251206/t_20251206_130843_058_u_3ce3c1a3-5ddb-406f-ac57-818c824c7afb.parquet
  fs/ecg_raw/day=20251206/t_20251206_130843_058_u_8972c51c-5dc2-4d75-8e63-335a856d8d49.parquet


In [5]:
import pandas as pd
import pyarrow.parquet as pq
import io

bucket_name = 'warehouse'
prefix = 'fs/ecg_raw/day=20251206/'

objects = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
dfs = []
if 'Contents' in objects:
    for obj in objects['Contents']:
        data = s3.get_object(Bucket=bucket_name, Key=obj['Key'])
        df = pq.read_table(io.BytesIO(data['Body'].read())).to_pandas()
        dfs.append(df)

df_all = pd.concat(dfs, ignore_index=True)
display(df_all)

timestamp device_name    ecg annotation
0      1765026476303   Person_01    0.1       None
1      1765026476305   Person_01  0.105       None
2      1765026476307   Person_01   0.11       None
3      1765026476309   Person_01   0.12       None
4      1765026476312   Person_01  0.135       None
...              ...         ...    ...        ...
19995  1765026476291   Person_01  0.115       None
19996  1765026476294   Person_01  0.105       None
19997  1765026476296   Person_01    0.1       None
19998  1765026476298   Person_01  0.105       None
19999  1765026476300   Person_01   0.09       None

[20000 rows x 4 columns]

In [1]:
from pyspark.sql import SparkSession

# Stop existing session
try:
    spark.stop()
except:
    pass


# Create Spark session with required JARs
spark = SparkSession.builder \
    .appName("Parquet S3 Test") \
    .config("spark.jars",
            "/home/ubuntu2020/Downloads/hadoop-aws-3.4.1.jar,"
            "/home/ubuntu2020/Downloads/bundle-2.40.0.jar,"
            "/home/ubuntu2020/Downloads/delta-spark_2.13-4.0.0.jar") \
    .config("spark.hadoop.fs.s3a.access.key", "admin") \
    .config("spark.hadoop.fs.s3a.secret.key", "password") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://127.0.0.1:9000") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .getOrCreate()

# Read Parquet from MinIO S3
# df = spark.read.parquet("s3a://warehouse/fs/ecg_raw/day=20251206/")
# df.show()


spark.read.parquet("s3a://warehouse/fs/ecg_raw/day=20251206/").show()




Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/07 10:23:09 WARN Utils: Your hostname, ubuntu2020-virtual-machine, resolves to a loopback address: 127.0.1.1; using 192.168.87.134 instead (on interface ens33)
25/12/07 10:23:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/12/07 10:23:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/07 10:23:24 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.htm

+-------------+-----------+-----+----------+
|    timestamp|device_name|  ecg|annotation|
+-------------+-----------+-----+----------+
|1765026476303|  Person_01|  0.1|      NULL|
|1765026476305|  Person_01|0.105|      NULL|
|1765026476307|  Person_01| 0.11|      NULL|
|1765026476309|  Person_01| 0.12|      NULL|
|1765026476312|  Person_01|0.135|      NULL|
|1765026476314|  Person_01| 0.12|      NULL|
|1765026476316|  Person_01| 0.11|      NULL|
|1765026476319|  Person_01|0.095|      NULL|
|1765026476322|  Person_01|  0.1|      NULL|
|1765026476324|  Person_01|0.095|      NULL|
|1765026476326|  Person_01|  0.1|      NULL|
|1765026476329|  Person_01| 0.11|      NULL|
|1765026476331|  Person_01| 0.11|      NULL|
|1765026476334|  Person_01|0.115|      NULL|
|1765026476336|  Person_01|0.115|      NULL|
|1765026476339|  Person_01|0.095|      NULL|
|1765026476342|  Person_01| 0.08|      NULL|
|1765026476344|  Person_01|0.075|      NULL|
|1765026476346|  Person_01| 0.07|      NULL|
|176502647

In [17]:
from pyspark.sql import SparkSession

# Stop existing session
try:
    spark.stop()
except:
    pass


spark = SparkSession.builder \
    .appName("Delta + S3") \
    .config("spark.jars.packages", "io.delta:delta-core_2.13:4.0.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://127.0.0.1:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "admin") \
    .config("spark.hadoop.fs.s3a.secret.key", "password") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.sql.warehouse.dir", "s3a://warehouse/") \
    .getOrCreate()

# Read Parquet from MinIO S3
# df = spark.read.parquet("s3a://warehouse/fs/ecg_raw/day=20251206/")
# df.show()


spark.read.parquet("s3a://warehouse/fs/ecg_raw/day=20251206/").show()




+-------------+-----------+-----+----------+
|    timestamp|device_name|  ecg|annotation|
+-------------+-----------+-----+----------+
|1765026476303|  Person_01|  0.1|      NULL|
|1765026476305|  Person_01|0.105|      NULL|
|1765026476307|  Person_01| 0.11|      NULL|
|1765026476309|  Person_01| 0.12|      NULL|
|1765026476312|  Person_01|0.135|      NULL|
|1765026476314|  Person_01| 0.12|      NULL|
|1765026476316|  Person_01| 0.11|      NULL|
|1765026476319|  Person_01|0.095|      NULL|
|1765026476322|  Person_01|  0.1|      NULL|
|1765026476324|  Person_01|0.095|      NULL|
|1765026476326|  Person_01|  0.1|      NULL|
|1765026476329|  Person_01| 0.11|      NULL|
|1765026476331|  Person_01| 0.11|      NULL|
|1765026476334|  Person_01|0.115|      NULL|
|1765026476336|  Person_01|0.115|      NULL|
|1765026476339|  Person_01|0.095|      NULL|
|1765026476342|  Person_01| 0.08|      NULL|
|1765026476344|  Person_01|0.075|      NULL|
|1765026476346|  Person_01| 0.07|      NULL|
|176502647

In [6]:
from pyspark.sql import SparkSession

# Stop existing session
try:
    spark.stop()
except:
    pass


spark = SparkSession.builder \
    .appName("Delta + S3") \
    .config("spark.jars.packages", "io.delta:delta-core_2.13:4.0.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://127.0.0.1:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "admin") \
    .config("spark.hadoop.fs.s3a.secret.key", "password") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.sql.warehouse.dir", "s3a://warehouse/") \
    .getOrCreate()

# Read Parquet from MinIO S3
# df = spark.read.parquet("s3a://warehouse/fs/ecg_raw/day=20251206/")
# df.show()


df = spark.read.parquet("s3a://warehouse/fs/ecg_raw/day=20251206/")




In [7]:
df = spark.read.parquet("s3a://warehouse/fs/ecg_raw/day=20251206/")

In [8]:
from trino.dbapi import connect

conn = connect(
    host="localhost",
    port=8082,
    user="admin"
)
cur = conn.cursor()

# 1) Create Silver schema inside Delta catalog
cur.execute("""
CREATE SCHEMA IF NOT EXISTS delta.hyb_silver
""")
print("Silver schema ready.")

# 2) Create Silver table structure
cur.execute("""
CREATE TABLE IF NOT EXISTS delta.hyb_silver.ecg_silver (
    timestamp BIGINT,
    device_name VARCHAR,
    ecg DOUBLE,
    annotation VARCHAR
)
""")
print("Silver table ready.")

# 3) Show schema
cur.execute("DESCRIBE delta.hyb_silver.ecg_silver")
schema = cur.fetchall()

for col in schema:
    print(col)

Silver schema ready.
Silver table ready.
['timestamp', 'bigint', '', '']
['device_name', 'varchar', '', '']
['ecg', 'double', '', '']
['annotation', 'varchar', '', '']


In [13]:
df = spark.read.parquet("s3a://warehouse/fs/ecg_raw/day=20251206/")

# Convert DF to Python list-of-tuples
rows = [tuple(r) for r in df.collect()]
print("Loaded", len(rows), "rows from Spark")

from trino.dbapi import connect

conn = connect(
    host="localhost",
    port=8082,
    user="admin",
    catalog="delta",
    schema="hyb_silver"
)
cur = conn.cursor()

# ---- Build SQL INSERT VALUES ----
values_sql = ", ".join(
    f"({r[0]}, '{r[1]}', {r[2]}, {'NULL' if r[3] is None else repr(r[3])})"
    for r in rows
)

sql = f"""
INSERT INTO ecg_silver (timestamp, device_name, ecg, annotation)
VALUES {values_sql}
"""

print("Executing SQL...")
cur.execute(sql)
conn.commit()

print("Inserted", len(rows), "rows into delta.hyb_silver.ecg_silver")


Loaded 20000 rows from Spark
Executing SQL...
Inserted 20000 rows into delta.hyb_silver.ecg_silver


In [14]:
df = spark.read.parquet("s3a://warehouse/fs/ecg_raw/day=20251206/")
rows = [tuple(r) for r in df.collect()]
print("Total rows:", len(rows))

from trino.dbapi import connect

conn = connect(
    host="localhost",
    port=8082,
    user="admin",
    catalog="delta",
    schema="hyb_silver"
)
cur = conn.cursor()

batch_size = 500
batch_count = 0

for i in range(0, len(rows), batch_size):
    batch = rows[i:i + batch_size]
    
    # build SQL for this batch
    values_sql = ", ".join(
        f"({r[0]}, '{r[1]}', {r[2]}, {'NULL' if r[3] is None else repr(r[3])})"
        for r in batch
    )
    
    sql = f"""
    INSERT INTO ecg_silver (timestamp, device_name, ecg, annotation)
    VALUES {values_sql}
    """
    
    cur.execute(sql)
    conn.commit()
    
    batch_count += 1
    print(f"Inserted batch {batch_count} ({len(batch)} rows)")

print("All batches inserted successfully.")


Total rows: 20000
Inserted batch 1 (500 rows)
Inserted batch 2 (500 rows)
Inserted batch 3 (500 rows)
Inserted batch 4 (500 rows)
Inserted batch 5 (500 rows)
Inserted batch 6 (500 rows)
Inserted batch 7 (500 rows)
Inserted batch 8 (500 rows)
Inserted batch 9 (500 rows)
Inserted batch 10 (500 rows)
Inserted batch 11 (500 rows)
Inserted batch 12 (500 rows)
Inserted batch 13 (500 rows)
Inserted batch 14 (500 rows)
Inserted batch 15 (500 rows)
Inserted batch 16 (500 rows)
Inserted batch 17 (500 rows)
Inserted batch 18 (500 rows)
Inserted batch 19 (500 rows)
Inserted batch 20 (500 rows)
Inserted batch 21 (500 rows)
Inserted batch 22 (500 rows)
Inserted batch 23 (500 rows)
Inserted batch 24 (500 rows)
Inserted batch 25 (500 rows)
Inserted batch 26 (500 rows)
Inserted batch 27 (500 rows)
Inserted batch 28 (500 rows)
Inserted batch 29 (500 rows)
Inserted batch 30 (500 rows)
Inserted batch 31 (500 rows)
Inserted batch 32 (500 rows)
Inserted batch 33 (500 rows)
Inserted batch 34 (500 rows)
Inser

In [16]:
from pyspark.sql import SparkSession
from trino.dbapi import connect

# ==========================
# Spark session
# ==========================
spark = SparkSession.builder \
    .appName("Delta + S3 + Trino Pipeline") \
    .config("spark.jars.packages", "io.delta:delta-core_2.13:4.0.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://127.0.0.1:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "admin") \
    .config("spark.hadoop.fs.s3a.secret.key", "password") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .config("spark.sql.warehouse.dir", "s3a://warehouse/") \
    .getOrCreate()

raw_path = "s3a://warehouse/fs/ecg_raw/"
archive_base = "s3a://warehouse/fs/ecg_archive/"

# ==========================================
# Infer schema statically (REQUIRED)
# ==========================================
static_df = spark.read.parquet(raw_path)
schema = static_df.schema
print("Inferred schema:")
print(schema)


# ==========================================
# Move raw files after writing to Trino
# ==========================================
def move_raw_files():
    hconf = spark._jsc.hadoopConfiguration()
    fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(hconf)

    src = spark._jvm.org.apache.hadoop.fs.Path(raw_path)
    dst = spark._jvm.org.apache.hadoop.fs.Path(archive_base)

    if not fs.exists(dst):
        fs.mkdirs(dst)

    files = fs.listStatus(src)
    for f in files:
        oldp = f.getPath()
        newp = spark._jvm.org.apache.hadoop.fs.Path(dst, oldp.getName())
        fs.rename(oldp, newp)
        print("Moved:", oldp, "→", newp)


# ==========================================
# foreachBatch → write each microbatch to Trino
# ==========================================
def write_to_trino(batch_df, batch_id):

    if batch_df.count() == 0:
        return

    print(f"Processing batch {batch_id}...")

    rows = [tuple(r) for r in batch_df.collect()]

    conn = connect(
        host="localhost",
        port=8082,
        user="admin",
        catalog="delta",
        schema="hyb_silver"
    )
    cur = conn.cursor()

    batch_size = 500

    for i in range(0, len(rows), batch_size):
        batch = rows[i:i + batch_size]

        values_sql = ", ".join(
            f"({r[0]}, '{r[1]}', {r[2]}, {'NULL' if r[3] is None else repr(r[3])})"
            for r in batch
        )

        insert_sql = f"""
        INSERT INTO ecg_silver (timestamp, device_name, ecg, annotation)
        VALUES {values_sql}
        """

        cur.execute(insert_sql)
        conn.commit()

        print(f"Inserted {len(batch)} rows into Trino Silver.")

    move_raw_files()
    print(f"Batch {batch_id} archived.")


# ==========================================
# Streaming read from S3
# ==========================================
stream_df = spark.readStream \
    .format("parquet") \
    .schema(schema) \
    .load(raw_path)

query = stream_df.writeStream \
    .foreachBatch(write_to_trino) \
    .option("checkpointLocation", "s3a://warehouse/checkpoints/ecg_stream/") \
    .start()

query.awaitTermination()


25/12/07 10:58:50 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Inferred schema:
StructType([StructField('timestamp', StringType(), True), StructField('device_name', StringType(), True), StructField('ecg', StringType(), True), StructField('annotation', StringType(), True), StructField('day', IntegerType(), True)])


25/12/07 10:58:50 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


Processing batch 0...
Inserted 500 rows into Trino Silver.
Inserted 500 rows into Trino Silver.
Inserted 500 rows into Trino Silver.
Inserted 500 rows into Trino Silver.
Inserted 500 rows into Trino Silver.
Inserted 500 rows into Trino Silver.
Inserted 500 rows into Trino Silver.
Inserted 500 rows into Trino Silver.
Inserted 500 rows into Trino Silver.
Inserted 500 rows into Trino Silver.
Inserted 500 rows into Trino Silver.
Inserted 500 rows into Trino Silver.
Inserted 500 rows into Trino Silver.
Inserted 500 rows into Trino Silver.
Inserted 500 rows into Trino Silver.
Inserted 500 rows into Trino Silver.
Inserted 500 rows into Trino Silver.
Inserted 500 rows into Trino Silver.
Inserted 500 rows into Trino Silver.
Inserted 500 rows into Trino Silver.
Inserted 500 rows into Trino Silver.
Inserted 500 rows into Trino Silver.
Inserted 500 rows into Trino Silver.
Inserted 500 rows into Trino Silver.
Inserted 500 rows into Trino Silver.
Inserted 500 rows into Trino Silver.
Inserted 500 row

25/12/07 10:59:25 ERROR MicroBatchExecution: Query [id = 2a1cf133-64b9-40d1-917c-26dd02ba0180, runId = f019129a-5a33-4f7e-a2d6-5d7186867633] terminated with error
org.apache.spark.sql.execution.streaming.sources.ForeachBatchUserFuncException: [FOREACH_BATCH_USER_FUNCTION_ERROR] An error occurred in the user provided function in foreach batch sink. Reason: An exception was raised by the Python Proxy. Return Message: Traceback (most recent call last):
  File "/home/ubuntu2020/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 644, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
  File "/home/ubuntu2020/.local/lib/python3.10/site-packages/pyspark/sql/utils.py", line 165, in call
    raise e
  File "/home/ubuntu2020/.local/lib/python3.10/site-packages/pyspark/sql/utils.py", line 162, in call
    self.func(DataFrame(jdf, wrapped_session_jdf), batch_id)
  File "/tmp/ipykernel_759799/3787175543.py", line 94, in write_to_trino
    move_raw_files()
  F

StreamingQueryException: [STREAM_FAILED] Query [id = 2a1cf133-64b9-40d1-917c-26dd02ba0180, runId = f019129a-5a33-4f7e-a2d6-5d7186867633] terminated with exception: [FOREACH_BATCH_USER_FUNCTION_ERROR] An error occurred in the user provided function in foreach batch sink. Reason: An exception was raised by the Python Proxy. Return Message: Traceback (most recent call last):
  File "/home/ubuntu2020/.local/lib/python3.10/site-packages/py4j/clientserver.py", line 644, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
  File "/home/ubuntu2020/.local/lib/python3.10/site-packages/pyspark/sql/utils.py", line 165, in call
    raise e
  File "/home/ubuntu2020/.local/lib/python3.10/site-packages/pyspark/sql/utils.py", line 162, in call
    self.func(DataFrame(jdf, wrapped_session_jdf), batch_id)
  File "/tmp/ipykernel_759799/3787175543.py", line 94, in write_to_trino
    move_raw_files()
  File "/tmp/ipykernel_759799/3787175543.py", line 42, in move_raw_files
    if no